In [2]:
import pandas as pd
from geopy.distance import geodesic

In [3]:
import pandas as pd
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
houses = df[['id', 'lat', 'long']]
houses.head()

,id,lat,long
0,7129300520,47.5112,-122.257
1,6414100192,47.7210,-122.319
2,5631500400,47.7379,-122.233
3,2487200875,47.5208,-122.393
4,1954400510,47.6168,-122.045


In [71]:
amenities = pd.read_csv("king_county_amenities.csv")
amenities

,name,type,latitude,longitude
0,Tahlequah,ferry_terminal,47.332006,-122.507785
1,Vashon Island,ferry_terminal,47.510915,-122.463838
2,Fauntleroy,ferry_terminal,47.523180,-122.396493
3,Unknown,Unknown,47.444876,-121.426365
4,Navos West Seattle Campus,hospital,47.534445,-122.366224
...,...,...,...,...
166,Seattle University,university,47.609510,-122.316832
167,Bastyr University,college,47.730662,-122.249185
168,Lake Washington Institute of Technology,college,47.705005,-122.167330
169,Seattle Children's Hospital,hospital,47.663096,-122.282949


In [73]:
amenities = amenities[amenities['type'] != 'Unknown']

In [75]:
# Extract the unique values from the 'Type' column
unique_values_type = amenities['type'].unique()

# Convert to a list
unique_values_list = unique_values_type.tolist()

print("Unique values in 'Type':", unique_values_list)

Unique values in 'Type': ['ferry_terminal', 'hospital', 'university', 'bus_station', 'college']


In [77]:
hospital_df = amenities[amenities['type'].isin(['hospital'])]
print(hospital_df.shape)
hospital_df.head()

(30, 4)


,name,type,latitude,longitude
4,Navos West Seattle Campus,hospital,47.534445,-122.366224
22,Kaiser Permanente Redmond Medical Center at Ri...,hospital,47.673286,-122.129091
63,The Seattle Facial Plastic Surgery Center,hospital,47.609170,-122.324205
72,Evergreen Endodontics - Tooth Pain & Dental Em...,hospital,47.543012,-122.045073
80,St. Francis Hospital,hospital,47.292191,-122.326048


In [51]:
transport_df = amenities[amenities['type'].isin(['ferry_terminal', 'bus_station'])]
print(transport_df.shape)
transport_df.head()

(30, 4)


,name,type,latitude,longitude
0,Tahlequah,ferry_terminal,47.332006,-122.507785
1,Vashon Island,ferry_terminal,47.510915,-122.463838
2,Fauntleroy,ferry_terminal,47.523180,-122.396493
9,Totem Lake Freeway Station,bus_station,47.715427,-122.185670
12,Kingsgate Park & Ride,bus_station,47.716591,-122.187645


In [53]:
university_df = amenities[amenities['type'].isin(['university', 'college'])]
print(university_df.shape)
university_df.head()

(53, 4)


,name,type,latitude,longitude
4,Embry - Riddle Aeronautical University,university,47.470543,-122.233636
5,Cascadia College,university,47.760868,-122.192210
10,Seattle Institute of Oriental Medicine,college,47.678045,-122.323478
11,Divers Institute of Technology,college,47.647605,-122.340835
13,Gene Juarez Academy,college,47.320809,-122.304503


(30, 4)

In [57]:
import time

# Record start time
start_time = time.time()

# Function to calculate nearest amenity
def calculate_nearest_amenity(house, amenities, amenity_type):
    house_location = (house['lat'], house['long'])
    min_distance = float('inf')
    nearest_amenity = None
    
    for _, amenity in amenities.iterrows():
        amenity_location = (amenity['latitude'], amenity['longitude'])
        distance = geodesic(house_location, amenity_location).kilometers  
        
        if distance < min_distance:
            min_distance = distance
            nearest_amenity = {
                f'nearest_{amenity_type}_name': amenity['name'],
                f'nearest_{amenity_type}_type': amenity['type'],
                f'distance_km_{amenity_type}': distance
            }
    
    return nearest_amenity

# List of amenities
amenity_types = {
    'hospital': hospital_df,
    'university': university_df,
    'station': transport_df
}

# Loop through each amenity type and calculate the nearest one
for amenity_type, amenity_data in amenity_types.items():
    houses[f'nearest_{amenity_type}'] = houses.apply(
        lambda house: calculate_nearest_amenity(house, amenity_data, amenity_type),
        axis=1
    )

    # Expand the dictionary into separate columns for the amenity type
    houses = houses.join(
        pd.DataFrame(houses[f'nearest_{amenity_type}'].tolist(), index=houses.index)
    )
    
    # Drop the intermediate column
    houses = houses.drop(columns=[f'nearest_{amenity_type}'])

# Record end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = (end_time - start_time)/ 60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

houses.head()

/var/folders/zl/jns9v8097kg7qj7p4fn603jc0000gn/T/ipykernel_46754/2562532878.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses[f'nearest_{amenity_type}'] = houses.apply(


Elapsed time: 7.71 minutes


,id,lat,long,nearest_hospital_name,nearest_hospital_type,distance_km_hospital,nearest_university_name,nearest_university_type,distance_km_university,nearest_station_name,nearest_station_type,distance_km_station
0,7129300520,47.5112,-122.257,Cascade Behavioral Health,hospital,3.939132,University of Phoenix - Western Washington Campus,university,4.824314,South Renton Park and Ride,bus_station,5.401288
1,6414100192,47.7210,-122.319,University of Washington Medical Center - Nort...,hospital,1.559600,PIMA Medical Institute,college,1.820426,Aurora Village Transit Center,bus_station,6.178197
2,5631500400,47.7379,-122.233,Fairfax Hospital,hospital,2.955235,Bastyr University,college,1.456564,Kingsgate Park & Ride,bus_station,4.146092
3,2487200875,47.5208,-122.393,Navos West Seattle Campus,hospital,2.523316,Highline Community College,college,3.690386,Fauntleroy Ferry Terminal Building,ferry_terminal,0.338328
4,1954400510,47.6168,-122.045,Swedish Hospital,hospital,7.773918,Central Washington University - Sammamish,university,1.108881,Overlake Park & Ride,bus_station,7.093464


In [63]:
# Save the results to a CSV file
houses.to_csv("house_distance_amenities.csv", index=False)